In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load training and test data
# Read TSV files, specifying tab separator and no header
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Encode labels: ham=0, spam=1
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Extract features and labels
train_messages = train_df['message'].values
train_labels = train_df['label'].values
test_messages = test_df['message'].values
test_labels = test_df['label'].values

# Tokenize text data
# Create a TextVectorization layer to convert text to integer sequences
vocab_size = 10000  # Maximum number of unique words
max_length = 50     # Maximum length of each message
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=max_length
)

# Adapt vectorizer to training messages to build vocabulary
vectorizer.adapt(train_messages)

# Build the neural network model
model = keras.Sequential([
    # Text vectorization layer to convert text to sequences
    vectorizer,
    # Embedding layer to learn word representations
    keras.layers.Embedding(vocab_size, 16, mask_zero=True),
    # Bidirectional LSTM to capture context in text
    keras.layers.Bidirectional(keras.layers.LSTM(16)),
    # Dense layer for classification
    keras.layers.Dense(16, activation='relu'),
    # Output layer: sigmoid for binary classification (0=ham, 1=spam)
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',  # Adam optimizer for efficient training
    loss='binary_crossentropy',  # Loss for binary classification
    metrics=['accuracy']  # Track accuracy during training
)

# Train the model
model.fit(
    train_messages,
    train_labels,
    epochs=10,  # Train for 10 epochs to converge
    validation_data=(test_messages, test_labels),
    batch_size=32,  # Batch size for training
    verbose=0  # Suppress output for cleaner notebook
)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

# Function to predict messages based on model
def predict_message(pred_text):
    # Convert input text to a TensorFlow string tensor
    pred_text = tf.convert_to_tensor([pred_text], dtype=tf.string)
    # Get model prediction (probability of spam)
    prediction = model.predict(pred_text, verbose=0)[0][0]
    # Determine label based on probability threshold (0.5)
    label = 'spam' if prediction >= 0.5 else 'ham'
    # Return [probability, label]
    return [float(prediction), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
